<a href="https://colab.research.google.com/github/ravi-kr/DeepLearning/blob/master/RNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM


Using TensorFlow backend.


In [0]:
# Training parameters.
batch_size = 32
num_classes = 10
epochs = 5

In [0]:
# Embedding dimensions.
row_hidden = 128
col_hidden = 128

In [4]:

# The data, split between train and test sets.
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [5]:
# Reshapes data to 4D for Hierarchical RNN.

# HRNNs can learn across multiple levels of temporal hierarchy over a complex sequence. Usually, the first recurrent layer of an HRNN encodes a sentence
# (e.g. of word vectors) into a sentence vector. The second recurrent layer then encodes a sequence of such vectors (encoded by the first layer) into a 
# document vector. This document vector is considered to preserve both the word-level and sentence-level structure of the context.
 
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = X_train.shape[1:]

# 4D input.
X = Input(shape=(row, col, pixel))


In [0]:
# Encodes a row of pixels using TimeDistributed Wrapper.
# TimeDistributed. This wrapper applies a layer to every temporal slice of an input. The input should be at least 3D, and the dimension of index one will be 
# considered to be the temporal dimension. Consider a batch of 32 samples, where each sample is a sequence of 10 vectors of 16 dimensions

encoded_rows = TimeDistributed(LSTM(row_hidden))(X)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)




In [0]:
# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(X, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# Training.
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))



In [0]:
# Evaluation.
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])